In [1]:
# Tools
import pandas as pd
import pickle
import re
from difflib import get_close_matches
from tqdm import tqdm
tqdm.pandas(desc = 'Fuzzy matching')

In [2]:
# Define the fuzzy merging method for dataframes
def fuzzymerge(self, right, on = None, left_on = None, right_on = None, 
               how = 'left', cutoff = 0):        
    '''Fuzzy merging. Same as standard pandas merge, but the `on` columns
    do not have to match exactly. Parameter `cutoff` controls this, e.g.: 
         * If cutoff = 0 (default), a match is always found.
         * If cutoff = 1, only a perfect match is acceptable.'''

    # Preliminary handling of `on` parameters
    if on is not None:
        left_on, right_on = on, on
    elif left_on is None or right_on is None:
        raise ValueError('Either `on`, or both `left_on` '
                         'and `right_on`, must be specified.')
    if type(left_on) != str or type(right_on) != str:
        raise TypeError('Must fuzzymerge on a single '
                        'column name in each frame.')

    # Fuzzy matching function
    right_values = right[right_on].unique()
    def fm(s):
        ret = get_close_matches(s, right_values, 1, cutoff)
        try: return ret[0]
        except: return None

    # Build intermediate dataframe for matching
    matches = self[[left_on]].drop_duplicates()
    matches['__matchcol__'] = matches[left_on].progress_apply(fm)        
    matches.dropna(subset = ['__matchcol__'], inplace = True)

    # Return results
    return self.merge(matches, on = left_on, how = how).merge(
        right, how = how, 
        left_on = '__matchcol__', 
        right_on = right_on
    ).drop('__matchcol__', 1)        

# Add fuzzy merging capability to all dataframes
pd.DataFrame.fuzzymerge = fuzzymerge

In [3]:
# Get the summoning comparison table
try:
    summ = pd.read_pickle('summ.pkl')
except:
    summ = pd.read_html(
        'https://www.el-wiki.net/Summoning_comparison_table',
        header = 0
    )[0]
    summ.to_pickle('summ.pkl')
    
# Process the ingredients list for each summonable creature
ingredients = (
    summ.Ingredients.str
    .extractall(r'(\d+)\s+([^,]+)')
    .reset_index('match').drop('match', 1)
)
ingredients.columns = ['quantity', 'ingredient']

In [6]:
# Get the current prices for all items
prices = pd.read_html(
    'http://greypal.el-fd.org/cgi-bin/querybot?action=Buy&item=.%2B',
    header = 0
)[0]
prices['Price'] = prices.Price.apply(lambda p: float(p.replace('gc', '')))

# Find the best price of mana potion in particular
pot_price = prices[prices.Item == 'Potion of Spirit Restoration'].Price.min()

In [39]:
# Create more complete comparison table using ingredient prices
full_summ = summ.join(ingredients)[[
    'Name', 'Ingredients', 'Level', 'BaseExp', 'ManaUsed', 
    'quantity', 'ingredient'
]]
full_summ = full_summ.fuzzymerge(
    prices.groupby('Item', as_index = False).min()[['Item','Price']],
    left_on = 'ingredient', right_on = 'Item'
).fuzzymerge(
    pd.read_pickle('weights.pkl'),
    left_on = 'ingredient', right_on = 'WikiPage'
)
full_summ['Price'] *= full_summ.quantity.astype(int)
full_summ['Weight'] *= full_summ.quantity.astype(int)

# Get total ingredient price per summonable creature
full_summ = full_summ.groupby(
    ['Name', 'Ingredients', 'Level', 'BaseExp', 'ManaUsed'],
    as_index = False
).agg({'Price': 'sum', 'Weight': 'sum', 
       'Stackable': lambda s: 'no' if 'no' in s else 'yes'})

# Add the projected price of mana restoration
def mana_convert(m):
    try: return float(m)
    except: return 0.0 
    
full_summ['Price'] += (full_summ.ManaUsed.apply(mana_convert) 
                       / 20 * pot_price)

# Price per unit of experience for each creature
full_summ['PricePerExp'] = full_summ.Price / full_summ.BaseExp
full_summ['WeightPerExp'] = full_summ.Weight / full_summ.BaseExp

Fuzzy matching: 100%|██████████████████████████████████████████████████████████████████| 42/42 [00:08<00:00,  5.31it/s]


In [46]:
# View the results
pd.set_option('display.max_colwidth', 999999)
full_summ.sort_values('PricePerExp').drop([
    'BaseExp', 'Price', 'Stackable'
], 1)

,Name,Ingredients,Level,ManaUsed,Weight,PricePerExp,WeightPerExp
38,Tank Rabbit,"1 Titanium Bar, 1 Brown Rabbit Fur, 1 Life Essence, 1 Bones, 1 Raw Meat",58,63,5,0.725375,0.025000
39,Wolf,"1 Wolf Fur, 2 Life Essence, 3 Bones, 2 Raw Meat",32,33,11,0.913750,0.183333
14,Female Goblin,"1 Iron Sword, 2 Life Essence, 3 Bones, 3 Raw Meat",41,46,15,1.044667,0.050000
8,Boar,"2 Life Essence, 3 Bones, 4 Raw Meat",26,29,9,1.074500,0.180000
2,Armed Goblin,"1 Iron Broad Sword, 2 Life Essence, 3 Bones, 3 Raw Meat",47,52,15,1.170125,0.037500
6,Black Bear,"1 Bear Fur, 3 Life Essence, 5 Bones, 5 Raw Meat",45,50,22,1.233333,0.244444
13,Falcon,"10 Falcon Feather, 20 Life Essence, 3 Bones, 2 Raw Meat",60,65,35,1.275417,0.116667
4,Armed Skeleton,"1 Steel Long Sword, 8 Bones",47,52,14,1.308600,0.028000
20,Hawk,"10 Hawk Feather, 3 Bones, 20 Life Essence, 2 Raw Meat",62,67,35,1.356797,0.109375
27,Polar Bear,"1 Polar Bear Fur, 3 Life Essence, 5 Bones, 5 Raw Meat",40,48,22,1.580625,0.275000
